In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, ShuffleSplit
import networkx as nx

import itertools

from pprint import pprint

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [2]:
import tensorflow as tf

In [3]:
from common.adjacency_list_to_graph import build_graph
from common.calculate_spring_rank import calculate_spring_rank
from common.graph_to_matrix import build_matrix

# Statistical significance

This page shows and explains the process of finding out SpringRank ranks statistical significance, that are extremely important to get really impressive results during genesis generation process (ahem, during the next few months).

# Table of contents
1. Building graph of transactions
2. Inferring $\beta$
3. Inferring $c$
4. Cross-validation process

# Building graph
Throughout the process of model calibration we'll use only part of Ethereum graph. It is placed in file named below

In [4]:
transactions_df = pd.read_csv("./part_data", sep=" ", names=["from", "to", "value", "block"])
transactions_df["value"] = 1
transactions_df = transactions_df.sort_values("block")

In [5]:
def create_dataset(dataset, addresses=None):
    if addresses:
        dataset = dataset[dataset["from"].isin(addresses) & dataset["to"].isin(addresses)]
    return dataset.groupby(["from", "to"])["value"].sum().to_frame()

In [6]:
def find_ranks(dataset, alpha):
    edges = dataset["value"].to_dict()
    graph = build_graph(edges)
    nodes = list(graph)
    A = build_matrix(graph, nodes)
    iterations, raw_rank = calculate_spring_rank(A, alpha=alpha)
    
    rank = pd.DataFrame()
    rank["address"] = nodes
    rank["rank"] = raw_rank
    
    return rank.set_index("address")

# Inferring $\beta$

SpringRank generative model includes formulas to find $E_{ij}$ - number of edges between $i$ and $j$ vertices in both directions, and $P_{ij}$ - proportion of edges that are going from vertex $i$ to vertex $j$. These formulas are going below:

$$P_{ij} = \frac{1}{1 + e^{-2\beta(s_i - s_j)}}, P_{ji} = 1 - P_{ij}$$

$$E_{ij} = c \exp{-\frac{\beta}{2}(s_i - s_j - 1)^2}$$

Except the calculated ranks, the described model has two parameters - temperature $\beta$ and density $c$

To find $\beta$, we'll treat ranks as constant values and apply maximum likelihood procedure described below:

$$L(A|s, \beta) = -\beta H(s) - M \log\sum_{i, j}\exp -\frac{\beta}{2}(s_i - s_j - 1)^2$$

There is a really large distance matrix $s_i - s_j$, so we'll do some sort of hack and calculate derivative not with TensorFlow graph mechanism, but by ourselves. Here it goes:

$$L'_{\beta}(A|s, \beta) 
= - H(s) + \frac{M}{2} \frac{1}{\sum_{i, j}e^{-\frac{\beta}{2}(s_i - s_j - 1)^2}}\sum_{i, j}e^{-\frac{\beta}{2}(s_i - s_j - 1)^2} (s_i - s_j - 1)^2$$

As a result, we'll get $\beta$ parameter that minimizes loss function $L$ in the presence of fixed ranks

In [8]:
s = tf.Session()

In [2]:
# It's a little bit hacky way cause of the size of distances matrix s_i - s_j

def predict_edges_sum(dataset, ranks, beta, c):
    addresses = set([a for a, _ in dataset.index] + [b for _, b in dataset.index])
    ranks = ranks.loc[addresses]["rank"].tolist()
    dataset_shape = len(ranks)
    batch_shape = 10000
    assert dataset_shape*batch_shape <= 10000000*1000
    ranks_tensor = tf.placeholder(tf.float64, shape=[None, ])
    dataset = tf.data.Dataset.from_tensor_slices(ranks_tensor)
    dataset = dataset.batch(batch_shape)
    dataset = dataset.map(lambda x: tf.reduce_sum(c*tf.exp(-beta / 2. * (tf.transpose([x]) - ranks_tensor - 1) ** 2)))

    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()

    s.run(iterator.initializer, {ranks_tensor: ranks})
    total_sum = 0
    count = 0
    
    while True:
        count += 1
        print("{} of {}".format(count, dataset_shape // batch_shape), end="\r")
        try:
            total_sum += s.run(next_element)
        except tf.errors.OutOfRangeError:
            break
    return total_sum / 2

In [10]:
def calculate_edges_sum_derivative(dataset, ranks, beta, c):
    addresses = set([a for a, _ in dataset.index] + [b for _, b in dataset.index])
    ranks = ranks.loc[addresses]["rank"].tolist()
    dataset_shape = len(ranks)
    batch_shape = 10000
    assert dataset_shape*batch_shape <= 10000000*1000
    ranks_tensor = tf.placeholder(tf.float64, shape=[None, ])
    dataset = tf.data.Dataset.from_tensor_slices(ranks_tensor)
    dataset = dataset.batch(batch_shape)
    
    def expression(x):
        ranks_difference = (tf.transpose([x]) - ranks_tensor - 1) ** 2
        return tf.reduce_sum(c * tf.exp(-beta / 2. * ranks_difference) * ranks_difference)
        
    dataset = dataset.map(expression)

    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()

    s.run(iterator.initializer, {ranks_tensor: ranks})
    total_sum = 0
    count = 0
    
    while True:
        count += 1
        print("{} of {}".format(count, dataset_shape // batch_shape), end="\r")
        try:
            total_sum += s.run(next_element)
        except tf.errors.OutOfRangeError:
            break
    return total_sum / 2

In [11]:
def calculate_energy(dataset, ranks):
    dataset["from_rank"] = ranks["rank"].loc[[a for a, _ in dataset.index]].tolist()
    dataset["to_rank"] = ranks["rank"].loc[[b for _, b in dataset.index]].tolist()
    dataset["energy"] = (dataset["from_rank"] - dataset["to_rank"] - 1) ** 2
    return (dataset["value"] * dataset["energy"]).sum() / 2

def infer_temperature(dataset, ranks):
    H = calculate_energy(dataset, ranks)
    M = float(dataset["value"].sum())
    beta_variable = tf.Variable(3.7268029290713383, name="beta", dtype=tf.float64)
    edges_sum_placeholder = tf.placeholder(tf.float64, shape=(None))
    edges_sum_derivative_placeholder = tf.placeholder(tf.float64, shape=(None))
    
    derivative = - H + (M / 2) * (1 / edges_sum_placeholder) * edges_sum_derivative_placeholder
    
    @tf.custom_gradient
    def loss_function(x):
        loss = - H * x - M * edges_sum_placeholder
        def grad(dy):
            return dy * derivative
        return loss, grad
    
    loss = loss_function(beta_variable)
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    s.run(tf.global_variables_initializer())
    for i in tqdm_notebook(range(0, 100)):
        try:
            beta = s.run(beta_variable)
            edges_sum = predict_edges_sum(dataset, ranks, beta=beta, c=1)
            edges_sum_derivative = calculate_edges_sum_derivative(dataset, ranks, beta=beta, c=1)
            variables = {
                edges_sum_placeholder: edges_sum, 
                edges_sum_derivative_placeholder: edges_sum_derivative
            }
            s.run(optimizer, variables)
            print("Loss:", s.run(loss, variables))
            print("Derivative:", s.run(derivative, variables))
            print("Beta:", s.run(beta_variable))
        except tf.errors.OutOfRangeError:
            break
    return s.run(beta_variable)

In [ ]:
infer_temperature(train, ranks)

# Inferring $c$
We'll infer generative model density simply by dividing the sum of predicted edges by the sum of edges that are presented in train dataset.

In [29]:
def infer_density(dataset, ranks, beta):
    edges_sum = predict_edges_sum(dataset, ranks, beta, 1)
    return dataset["value"].sum() / edges_sum

# Cross-validation
Generative model parameters are inferred, so we can try to cross-validate results. High value of metric after cross-validation procedure will be an indicator that we chose $\alpha, \beta, c$ parameters properly, and calcucated ranks are statistically significant

To start a process, we need to calculate edges number between vertices $E_{ij}$ and their directions $p_{ij}$, and to measure the performance of our model. All calculations we should do over the chunked dataset, each chunk will play a role of test part, the rest of dataset will be the train part. The SpringRank paper describes multigraph accuracy as a measure of model performance, formula goes below:
$$\sigma_a = 1 - \frac{1}{2M}\sum_{i,j}|{A_{ij} - (A_{ij} + A_{ji})P_{ij}}|$$
Where $M$ - overall number of edges.

It's important to say that we can deal with multigraph accuracy much faster on sparse graph with skipping empty edges in one and in both directions. We can treat both cases in a way described below:

1. If $A_{ij} \neq 0 $ and $A_{ji} = 0$: $E_{ji}(1 - P_{ji}) + |A_{ji} - E_{ji}P_{ji}|$
2. If $A_{ij} = 0$ and $A_{ji} = 0$: $E_{ij}$

So the accuracy of predictions for such edges can be described by this formula:
$$\sum E_{ij} - \frac{\sum_{(i,j), (j,i) \in G} E_{ij}}{2} - \sum_{(i,j) \in G, (j, i) \in G} E_{ij}$$

In [30]:
def predict_edges(dataset, ranks, beta, c):
    dataset["from_rank"] = ranks.loc[[a for a, _ in dataset.index]]["rank"].tolist()
    dataset["to_rank"] = ranks.loc[[b for _, b in dataset.index]]["rank"].tolist()
    dataset["direction_probability"] = (1 / (1 + np.exp(-2 * beta * (dataset["from_rank"] - dataset["to_rank"]))))
    dataset["number_of_edges"] = c * np.exp(- beta / 2 * (dataset["from_rank"] - dataset["to_rank"] - 1) ** 2)
    return dataset["direction_probability"].tolist(), dataset["number_of_edges"].tolist(), predict_edges_sum(dataset, ranks, beta, c)

In [32]:
def accuracy(dataset, direction, edges, edges_sum):
    accuracy_dataset = dataset.copy()
    accuracy_dataset = accuracy_dataset.merge(accuracy_dataset.reset_index(), left_index=True, right_on=["to", "from"], how="left", suffixes=('', '_reversed'))
    accuracy_dataset["not_paired"] = 0
    accuracy_dataset.loc[np.isnan(accuracy_dataset["value_reversed"]), "not_paired"] = 1
    accuracy_dataset["direction"] = direction
    accuracy_dataset["edges"] = edges
    accuracy_dataset["prediction"] = accuracy_dataset["direction"] * accuracy_dataset["edges"]
    accuracy_dataset["error"] = np.abs(accuracy_dataset["value"] - accuracy_dataset["prediction"])
    accuracy_dataset["non_paired_error"] = accuracy_dataset["not_paired"] * accuracy_dataset["edges"] * (1 - accuracy_dataset["direction"])
    non_active_edges_sum = edges_sum - (accuracy_dataset["not_paired"] * accuracy_dataset["edges"]).sum() - ((1 - accuracy_dataset["not_paired"]) * accuracy_dataset["edges"]).sum() / 2
    return 1 - \
        (accuracy_dataset["error"].sum() + accuracy_dataset["non_paired_error"].sum() + non_active_edges_sum) / \
        (accuracy_dataset["value"].sum() + edges_sum) 

In [33]:
test_df = pd.DataFrame()
test_df["from"] = ["0x1", "0x2", "0x3"]
test_df["to"] = ["0x2", "0x1", "0x4"]
test_df["value"] = [1, 2, 1]
test_df = test_df.set_index(["from", "to"])

In [ ]:
alphas = [0]
# alphas = np.logspace(-2, 2, 5)
betas = [4922.40704411323]
# betas = [288808]
split = ShuffleSplit(n_splits=5)
# split = TimeSeriesSplit(n_splits=5)

train_metrics = {}
test_metrics = {}

train_index, test_index = next(split.split(transactions_df))
# for train_index, test_index in :
train = create_dataset(transactions_df.loc[train_index])
print("Train size: {} samples".format(train.shape[0]))
train_addresses = list([a for a, _ in train.index] + [b for _, b in train.index])
test = create_dataset(transactions_df.loc[test_index], addresses=train_addresses)
print("Test size: {} samples".format(test.shape[0]))
for alpha in alphas:
    ranks = find_ranks(train, alpha=alpha)
    for beta in betas:
        c = infer_density(train, ranks, beta=beta)
        train_directions, train_edges, train_sum = predict_edges(train, ranks, beta=beta, c=c)
        print(beta * c, train_sum)
        test_directions, test_edges, test_sum = predict_edges(test, ranks, beta=beta, c=c)
        print(train_sum, test_sum)
        train_metrics[(alpha, beta, c)] = train_metrics.get((alpha, beta, c), []) + [accuracy(train, train_directions, train_edges, train_sum)]
        test_metrics[(alpha, beta, c)] = test_metrics.get((alpha, beta, c), []) + [accuracy(test, test_directions, test_edges, test_sum)]
        print("Train accuracy: ", train_metrics[(alpha, beta, c)][-1])
        print("Test accuracy: ", test_metrics[(alpha, beta, c)][-1])
pprint(train_metrics)
pprint(test_metrics)

In [155]:
train_metrics_df = pd.DataFrame().from_dict(train_metrics).mean().to_frame().reset_index().rename(columns={"level_0": "alpha", "level_1": "beta", 0: "accuracy", "level_2": "c"})
test_metrics_df = pd.DataFrame().from_dict(test_metrics).mean().to_frame().reset_index().rename(columns={"level_0": "alpha", "level_1": "beta", 0: "accuracy", "level_2": "c"})

In [156]:
test_metrics_df["alpha_log"] = np.log(test_metrics_df["alpha"])
train_metrics_df["alpha_log"] = np.log(train_metrics_df["alpha"])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


Some cross-validation statistics will be plotted there. Now these cells are empty

In [ ]:
plt.plot(train_metrics_df.groupby("alpha_log")["accuracy"].mean(), label="train")
plt.plot(test_metrics_df.groupby("alpha_log")["accuracy"].mean(), label="test")
plt.legend()

In [ ]:
plt.plot(train_metrics_df.groupby("beta")["accuracy"].max(), label="train")
plt.plot(test_metrics_df.groupby("beta")["accuracy"].max(), label="test")
plt.legend()

In [ ]:
plt.plot(train_metrics_df.groupby("c")["accuracy"].max(), label="train")
plt.plot(test_metrics_df.groupby("c")["accuracy"].max(), label="test")
plt.legend()